In [1]:
import sys
sys.path.append('../')
import wrangle

In [2]:
import nlp

In [3]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [4]:
import pyLDAvis
import pyLDAvis.sklearn

In [5]:
df, data_dict = wrangle.wrangle_data()

In [6]:
pyLDAvis.enable_notebook()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 78 columns):
job_title                         726 non-null category
job_taxo                          726 non-null category
job_id                            723 non-null category
job_conduct_res                   726 non-null int64
job_analyze_res                   726 non-null int64
job_buy_res_report                726 non-null int64
job_manage_res_proj               726 non-null int64
job_observe_res                   726 non-null int64
job_plan_res                      726 non-null int64
job_teach_res                     726 non-null int64
job_advocate_res                  726 non-null int64
job_hire_res_vendor               726 non-null int64
job_lead_res_team                 726 non-null int64
num_employees                     723 non-null category
num_researchers                   722 non-null category
primary_industry                  726 non-null category
types_res_used               

In [14]:
future_research = df.future_res.dropna().apply(nlp.basic_cleanic_clean)

In [15]:
future_research = future_research.apply(nlp.lemmatize)

In [ ]:
df.recommendations.dropna().apply(nlp.lemmatize)

## Process

In [16]:
word_count_matrix, count_vect = nlp.create_wordcount_matrix(future_research, max_df=0.3, min_df=2, ngram=(1,3))

In [ ]:
word_count_matrix

In [17]:
LDA = LatentDirichletAllocation(n_components=3, random_state=42)
LDA.fit(word_count_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [18]:
pyLDAvis.sklearn.prepare(LDA, word_count_matrix, count_vect)

/usr/local/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.096808  0.147788       1        1  38.557449
2     -0.188419  0.002743       2        1  30.852486
1      0.091610 -0.150531       3        1  30.590065, topic_info=    Category        Freq         Term       Total  loglift  logprob
371  Default  139.000000     research  139.000000  30.0000  30.0000
434  Default   69.000000        study   69.000000  29.0000  29.0000
90   Default   39.000000        diary   39.000000  28.0000  28.0000
459  Default   91.000000      testing   91.000000  27.0000  27.0000
91   Default   35.000000  diary study   35.000000  26.0000  26.0000
..       ...         ...          ...         ...      ...      ...
76    Topic3   11.314229         data   43.352559  -0.1588  -4.4350
29    Topic3    6.640091       better   12.828248   0.5260  -4.9680
454   Topic3    5.823614         team   14.422670   0.2776  -5.0992
266   Topic3    5.095950    moderated    9.396425   0.5726  -5.2326
444   Topic3    6.032945       survey   37.804648  -0.6507  -5.0639

[176 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.966467          ab
2         1  0.949014  ab testing
3         3  0.941622      access
6         3  0.941649  additional
10        1  0.217909    analysis
...     ...       ...         ...
536       2  0.252477        work
536       3  0.673271        work
537       2  0.946919     working
538       1  0.741231    workshop
538       2  0.211780    workshop

[197 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [11]:
LDA.transform(word_count_matrix)

array([[0.04865894, 0.48496094, 0.46638011],
       [0.33333333, 0.33333333, 0.33333333],
       [0.16682112, 0.66314436, 0.17003452],
       ...,
       [0.08755814, 0.82907687, 0.083365  ],
       [0.51182215, 0.45753248, 0.03064537],
       [0.02645341, 0.58808696, 0.38545963]])

In [12]:
nlp.find_top_documents_per_topic(LDA.transform(word_count_matrix), future_research, 3)

Top 3 Documents for Topic 0: 

Document 1
I'd love to just flat-out assemble a customer "council" database so that we can engage in more frequent and lighter-weight customer research sessions, a I've done at previous jobs, but there are major cultural hurdle - managing sales/channel partner relationship and mistrust, etc. This is still my major goal that I don't want to fully give up on, though.

Document 2
More contextual/observational studies, unmoderated studies, more co-design workshops, generally looking for opportunity to be more lean, quick, and agile with the process (leveraging existing insight from past research is a large part of that effort for us)

Document 3
more service design, design thinking learning proper SPSS quant method to complement qual method going to try out Delve for grounded theory analysis

Top 3 Documents for Topic 1: 

Document 1
User Experience Research (workshops, surveys, interviews, usability testing, benchmark, card sorting, etc.) and I would like to

In [ ]:
from textblob import TextBlob, Word

In [ ]:
word = 'studies'

In [ ]:
w = Word(word)
w.lemmatize()

In [ ]:
import nltk

In [ ]:
wnl = nltk.stem.WordNetLemmatizer()

In [ ]:
wnl.lemmatize(word)

In [ ]:
lemmas = [wnl.lemmatize(word) for word in text.split()]